In [1]:
import numpy as np 
import pandas as pd
import os, time

## Loading the data

In [2]:
#Loading the Data
eDay = pd.read_csv('Expected_EndOfDay_Positions.txt')
stDay = pd.read_csv ('Input_StartOfDay_Positions.txt')
transactions = pd.read_json('1537277231233_Input_Transactions.txt')

#### Checking the Data

In [3]:
eDay

,Instrument,Account,AccountType,Quantity,Delta
0,IBM,101,E,101000,1000
1,IBM,201,I,-101000,-1000
2,MSFT,101,E,4999750,-250
3,MSFT,201,I,-4999750,250
4,APPL,101,E,-1100,-11100
5,APPL,201,I,1100,11100
6,AMZN,101,E,-24850,-14850
7,AMZN,201,I,24850,14850
8,NFLX,101,E,100000000,0
9,NFLX,201,I,-100000000,0


In [4]:
stDay

,Instrument,Account,AccountType,Quantity
0,IBM,101,E,100000
1,IBM,201,I,-100000
2,MSFT,101,E,5000000
3,MSFT,201,I,-5000000
4,APPL,101,E,10000
5,APPL,201,I,-10000
6,AMZN,101,E,-10000
7,AMZN,201,I,10000
8,NFLX,101,E,100000000
9,NFLX,201,I,-100000000


In [5]:
transactions

,Instrument,TransactionId,TransactionQuantity,TransactionType
0,IBM,1,1000,B
1,APPL,2,200,S
2,AMZN,3,5000,S
3,MSFT,4,50,B
4,APPL,5,100,B
5,APPL,6,20000,S
6,AMZN,7,5000,S
7,MSFT,8,300,S
8,AMZN,9,200,B
9,APPL,10,9000,B


### Processing the Data

**Arranging the Data of transaction table to more readable form**

In [6]:
transData = transactions.groupby(['TransactionType','Instrument']).sum()
transData.reset_index(inplace=True)

In [7]:
transData

,TransactionType,Instrument,TransactionId,TransactionQuantity
0,B,AMZN,9,200
1,B,APPL,15,9100
2,B,IBM,1,1000
3,B,MSFT,4,50
4,S,AMZN,33,15050
5,S,APPL,8,20200
6,S,MSFT,8,300


**Function to get the Quantity from transaction table for buy or sell**


In [12]:
def getTransactionQuantity(instrument,transType):
    instData = transData[transData['Instrument'] == instrument]
    sell = instData[instData['TransactionType'] == 'S']['TransactionQuantity'] 
    buy = instData[instData['TransactionType'] == 'B']['TransactionQuantity']

    #Verifying if the quantity if is is zero
    if sell.shape[0] <= 0:
        sell = 0        
    if buy.shape[0] <= 0:
        buy = 0
        
    if transType == 'E':
        delta = float(buy) - float(sell)
    elif transType == 'I':
        delta = float(sell) - float(buy)
    else:
        return('Transaction type is inavlid')
    return delta


## Calculating the delta and updating the records to position table.
If Transaction Type =B ,

For AccountType=E, Quantity=Quantity + TransactionQuantity

For AccountType=I, Quantity=Quantity - TransactionQuantity

If Transaction Type =S ,

For AccountType=E, Quantity=Quantity - TransactionQuantity

For AccountType=I, Quantity=Quantity + TransactionQuantity


In [9]:
#using apply method with lambda function to call the mehtod for getting delta value of quantity
stDay['Delta'] = stDay[['Instrument','AccountType']].apply(lambda x : getTransactionQuantity(*x),axis=1)
stDay['Quantity'] = stDay['Quantity'] + stDay['Delta']


**Verifying the new Data**

In [10]:
#newly updated data
stDay

,Instrument,Account,AccountType,Quantity,Delta
0,IBM,101,E,101000.0,1000.0
1,IBM,201,I,-101000.0,-1000.0
2,MSFT,101,E,4999750.0,-250.0
3,MSFT,201,I,-4999750.0,250.0
4,APPL,101,E,-1100.0,-11100.0
5,APPL,201,I,1100.0,11100.0
6,AMZN,101,E,-24850.0,-14850.0
7,AMZN,201,I,24850.0,14850.0
8,NFLX,101,E,100000000.0,0.0
9,NFLX,201,I,-100000000.0,0.0


In [14]:
#End of the day data
eDay

,Instrument,Account,AccountType,Quantity,Delta
0,IBM,101,E,101000,1000
1,IBM,201,I,-101000,-1000
2,MSFT,101,E,4999750,-250
3,MSFT,201,I,-4999750,250
4,APPL,101,E,-1100,-11100
5,APPL,201,I,1100,11100
6,AMZN,101,E,-24850,-14850
7,AMZN,201,I,24850,14850
8,NFLX,101,E,100000000,0
9,NFLX,201,I,-100000000,0


## Instrument with Lowest and Largest Net Transaction Volume

·         At the end of the process find instruments with largest and lowest net transaction volumes for the day. (net volume is change in positions from start of day positions to end of day positions)

 

In [49]:
stDay[stDay['Delta'] == stDay['Delta'].min()]

,Instrument,Account,AccountType,Quantity,Delta
6,AMZN,101,E,-24850.0,-14850.0


In [50]:
transData[transData['Delta'] == stDay['Delta'].max()]

,Instrument,Account,AccountType,Quantity,Delta
7,AMZN,201,I,24850.0,14850.0
